# Домашнее задание 3. VAE + NF + VAPNEV

### Загрузка данных
В данном задании вам предстоит снова работать с CelebA

Решением домашки является архив с использованными тетрадками/модулями, а так же .pdf файл с отчетом по проделанной работе по каждому пункту задачи. В нем необходимо описать какие эксперименты вы производили чтобы получить результат который вы получили, а так же обосновать почему вы решили использовать штуки которые вы использовали (например, дополнительные лоссы для стабилизации, разные виды потоков, разные хаки для вае)



In [ ]:
%pip install numba
%pip install pytorch-fid-wrapper
%pip install wandb
%pip install -U --no-cache-dir gdown --pre
!git clone https://github.com/HSE-LAMBDA/DeepGenerativeModels.git
!gdown --id 17lt5GBYn17jMXbP1riwpbrW0byY2UY70
# import gdown
# gdown.download(id="17lt5GBYn17jMXbP1riwpbrW0byY2UY70", quiet=False)
!unzip -q img_align_celeba.zip* 
!curl https://raw.githubusercontent.com/taki0112/StarGAN-Tensorflow/master/dataset/celebA/list_attr_celeba.txt 2> /dev/null | tail +2 > list_attr_celeba.txt
!cat list_attr_celeba.txt > img_align_celeba/list_attr_celeba.txt

In [ ]:
!wandb login 

In [ ]:
from DeepGenerativeModels.utils.datasets.celeba import CelebADataset
from datetime import datetime
from matplotlib import rcParams
import pytorch_fid_wrapper as pfw
from torch import Tensor
from torch import nn
from torch.optim.lr_scheduler import LambdaLR
from torchvision import transforms
from torchvision.models import inception_v3
from torchvision.utils import make_grid
from tqdm import tqdm
from tqdm.auto import trange, tqdm
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import wandb
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
%matplotlib inline
rcParams['figure.figsize'] = (20, 10)
cwd = os.getcwd()
torch.manual_seed(142)
np.random.seed(142)

In [ ]:
t_normalize = lambda x: x * 2 - 1
t_invnormalize = lambda x: (x + 1) / 2
transform=transforms.Compose([
    transforms.Resize(64),
    transforms.CenterCrop(64),
    transforms.ToTensor(),
    transforms.RandomApply([transforms.functional.hflip]),
    transforms.Lambda(t_normalize),
])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
dataset = CelebADataset(root_dir=cwd, transform=transform)

num_workers = 0 if device.type == "cuda" else 2

pin_memory = True if device.type == "cuda" else False

dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=16,
    num_workers=num_workers,
    pin_memory=pin_memory,
    shuffle=True,
)

In [ ]:
def make_grid_np(images, nrow=8):
    return make_grid(
        [t_invnormalize(img) for img in images],
        pad_value=1,
        padding=3,
        nrow=nrow
    ).permute(1, 2, 0).numpy()

def show_images(images, title=None, nrow=8):
    plt.imshow(make_grid_np(images,nrow=nrow))
    plt.axis("off")
    if title is not None:
        plt.title(title)
    plt.show()

### Визуализация датасета

In [ ]:
show_images([dataset[np.random.randint(len(dataset))][0] for i in range(32)], nrow=8)

### Задача 1 (4/10 балла). Построить и обучить нормпоток на CelebA

Здесь нужно обучить нормпоток до нормального (трешхолды будут позже) качества, померить FID и Negative Log Likelihood и запомнить для будущего сравнения

Внутри потока можно использовать все что вы хотите, Coupling/Autoregressive/Linear слои, любые трансформации и все что вам приходит в голову. Но все что вы используете - напишите сами, без копипаста.

Можно использовать как и сверточные потоки (будут лучше, но сложнее писать), так и линейные (будут хуже), развернув селебу в один вектор.

В принципе можно использовать тот код который уже есть в репозитории курса, но лучше написать свой

### Задача 2 (2/10 балла). Построить и обучить VAE на CelebA

Здесь нужно обучить VAE до нормального качества, померить FID и запомнить для будущего сравнения. 

В принципе можно использовать тот код который уже есть в репозитории курса, но лучше написать свой

Ради интереса, ваше вае тоже можно потестировать на маленьких датасетах


In [ ]:
class Encoder(nn.Module):
    def __init__(self, img_size=64, latent_size=512, start_channels=16, downsamplings=5):
        super().__init__()
        blocks = [nn.Conv2d(in_channels=3, out_channels=start_channels, kernel_size=1, stride=1, padding=0)]
        in_channels = start_channels
        for i in range(downsamplings):
            blocks += [
                nn.Conv2d(in_channels=in_channels, out_channels=in_channels * 2, kernel_size=3, stride=2, padding=1),
                nn.BatchNorm2d(in_channels * 2),
                nn.LeakyReLU()
            ]
            in_channels *= 2
            img_size //= 2
        blocks += [
            nn.Flatten(),
            nn.Linear(in_features=in_channels * img_size ** 2, out_features=2 * latent_size)
        ]
        self.network = nn.Sequential(*blocks)
        self.N = torch.distributions.Normal(0, 1)

    def forward(self, x):
        x = self.network(x)
        mu, sigma = x.split(x.size(1) // 2, dim=1)
        sigma = torch.exp(sigma)
        return mu + sigma * self.N.sample(mu.shape).to(device), (mu, sigma)


class Decoder(nn.Module):
    def __init__(self, img_size=64, latent_size=512, end_channels=16, upsamplings=5):
        super().__init__()
        in_channels = end_channels * (2 ** upsamplings)
        img_size //= 2 ** upsamplings
        blocks = [
            nn.Linear(in_features=latent_size, out_features=in_channels * img_size ** 2),
            nn.Unflatten(1, (in_channels, img_size, img_size)),
        ]
        for i in range(upsamplings):
            blocks += [
                nn.ConvTranspose2d(in_channels=in_channels, out_channels=in_channels // 2, kernel_size=4, stride=2,
                                   padding=1),
                nn.BatchNorm2d(in_channels // 2),
                nn.LeakyReLU()
            ]
            in_channels //= 2
            img_size *= 2
        blocks.append(nn.Conv2d(in_channels=in_channels, out_channels=3, kernel_size=1, stride=1, padding=0))
        blocks.append(nn.Tanh())
        self.network = nn.Sequential(*blocks)

    def forward(self, z):
        return self.network(z)

class VAE(nn.Module):
    def __init__(self, img_size=64, downsamplings=3, latent_size=120, down_channels=6, up_channels=9):
        super().__init__()
        self.encoder = Encoder(img_size, latent_size, down_channels, downsamplings)
        self.decoder = Decoder(img_size, latent_size, up_channels, downsamplings)

    def forward(self, x):
        x, (mu, sigma) = self.encoder(x)
        kld = 0.5 * (sigma ** 2 + mu ** 2 - 2 * torch.log(sigma + 1e-9) - 1)
        x = self.decoder(x)
        return x, kld

    def encode(self, x):
        return self.encoder(x)[0]

    def decode(self, z):
        return self.decoder(z)

    def save(self, name="model.pth"):
        torch.save(self.state_dict(), name)

    def load(self, name=None):
        self.load_state_dict(torch.load(name if name is not None else (__file__[:-7] + "model.pth")))
        self.eval()

In [ ]:
def calculate_vae_fid_score(dataloader, model, batches=100):
    with torch.no_grad():
        real_img = []
        fake_img = []
        for i, (batch, meta) in tqdm(enumerate(dataloader), total=batches):
            if i == batches:
                break
            real_img.extend(batch)
            fake_img.extend(vae.decoder(torch.randn_like(vae.encode(torch.zeros(dataloader.batch_size, 3, 64, 64).to(device)))))
        return pfw.fid(torch.stack(real_img).to(device), torch.stack(fake_img).to(device), device=device)

In [ ]:
wandb.init(
    project="generative-models-vae",

    config={
        "kl_factor": 5,
        "architecture": "VAE",
        "dataset": "CelebA",
        "epochs": 10,
        "batch_size": 16,
        "optimizer": "adam",
        "learning_rate": 0.001
    }
)

In [ ]:
vae = VAE().to(device)

In [ ]:
batch_size = wandb.config["batch_size"]
vae_optim = torch.optim.Adam(vae.parameters(), 0.001)


total_batch_id = 0
for epoch in range(wandb.config["epochs"]):
    total_batches = 0
    rec_loss_avg = 0
    kld_loss_avg = 0
    total_loss_avg = 0

    for i, batch in tqdm(enumerate(dataloader), total=(len(dataset) + batch_size) // batch_size):
        if i % 1000 == 0:
            with torch.no_grad():
                generated_file = f'generated{total_batch_id}.png'

                plt.imsave(generated_file, make_grid_np(vae.decoder(torch.randn_like(vae.encode(torch.zeros(32, 3, 64, 64).to(device)))).cpu()))

                artifact = wandb.Artifact('generated_image.png', type='image')
                artifact.add_file(generated_file)
                wandb.log_artifact(artifact)


                interpolated_file = f'generated{total_batch_id}.png'

                test_imgs_1 = torch.cat([dataset[i][0].unsqueeze(0) for i in (31, 34, 736, 1520, 34, 190, 42, 178)])
                test_imgs_2 = torch.cat([dataset[i][0].unsqueeze(0) for i in (731, 132, 51, 32, 98, 67, 127, 79)])
                z_1 = vae.encode(test_imgs_1.to(device))
                z_2 = vae.encode(test_imgs_2.to(device))
                x_int = []
                for i in range(9):
                    z = (i * z_1 + (8 - i) * z_2) / 8
                    x_int.append(vae.decode(z))
                x_int = torch.cat(x_int).cpu()
                plt.imsave(interpolated_file, make_grid_np([x_int[j*8+i] for i in range(8) for j in range(9)], nrow=9))

                artifact = wandb.Artifact('interpolated_image.png', type='image')
                artifact.add_file(interpolated_file)
                wandb.log_artifact(artifact)


        if (total_batch_id % 2000 == 0):
            model_file = f'model_{total_batch_id}.pth'
            vae.save(model_file)
            artifact = wandb.Artifact('model_weights', type='model')
            artifact.add_file(model_file)
            wandb.log_artifact(artifact)
            
            wandb.log({'fid': calculate_vae_fid_score(dataloader, vae, 100)}, step=total_batch_id)

        batch = batch[0]
        if len(batch) < batch_size:
            continue
        total_batches += 1
        x = batch.to(device)
        x_rec, kld = vae(x)
        img_elems = float(np.prod(list(batch.size())))
        kld_loss = kld.sum() / batch_size
        rec_loss = ((x_rec - x)**2).sum() / batch_size
        loss = rec_loss + wandb.config["kl_factor"] * kld_loss 
        vae_optim.zero_grad()
        loss.backward()
        vae_optim.step()
        kld_loss_avg += kld_loss.item()
        rec_loss_avg += rec_loss.item()
        total_loss_avg += loss.item()

        wandb.log({'kld_loss': kld_loss.item(), 'reconstruction_loss': rec_loss.item(), 'total_loss': loss.item()}, step=total_batch_id)
        total_batch_id += 1
        if i % 1000 == 0:
            print(rec_loss_avg / total_batches, kld_loss_avg / total_batches, total_loss_avg / total_batches)


    rec_loss, kld_loss = rec_loss_avg / total_batches, kld_loss_avg / total_batches
    wandb.log({'kld_loss_epoch': kld_loss, 'reconstruction_loss_epoch': rec_loss}, step=epoch)


Ссылка на запуск в WandB: https://wandb.ai/debnatkh/generative-models-vae/runs/xm2cpe8c?workspace=user-debnatkh

FID: 93.66

Пример сгенерированной картинки:

![image.png](https://storage.googleapis.com/wandb-artifacts-prod/wandb_artifacts/56528774/393045797/2c7605c8f9c8a11025a7fcef084bc8e4?Expires=1678466558&GoogleAccessId=wandb-production%40appspot.gserviceaccount.com&Signature=LZwokmzDWTK5eu7r1CVMRiFypmAIFAODGL6jeC2p08bQsPJiJgQWMKvxepF0D1zxgr7yb6qFsYpZGJQmiUecPts5AAHLVG0susUEyeVfAaRpGb3v4msTHW8tSUHsY%2FSdoI%2B1okSkPXodJ%2FPOoc%2BEgSP7eQ7gNrmKm5VdmijwVjs6pc5iLa%2FYCub73uZCu60OUl2PNmpnMgYLl2oRZztF7e0NOrPefUpOn4Ji0wiAkcSR%2BiZx4gsISFgerr1txUTu1KUZmfvUOjAt3BZI3uQqzFrNH51avmT8M3Jdwm37X08xyRJkxugb%2BXkIcXup0MnT3JuqP1ht61dT6BmHBQoWqg%3D%3D)

Линейная интерполяция между двумя точками датасета:

![image.png](https://storage.googleapis.com/wandb-artifacts-prod/wandb_artifacts/56528777/393045808/aac09c0cd73d302e089ab0c2a4425b92?Expires=1678466809&GoogleAccessId=wandb-production%40appspot.gserviceaccount.com&Signature=OqVo2WpXdihbGRUoih%2ByGPDv7za5DS4x8Ww7Q25BZfOsI6PPX9xpwdiiZAqfhNGa1%2B6nWtczCb988DwleDU5Px%2FWQxX%2FFk8T%2FJARKhv34A90rXfJd0OBQhR76%2FjYz9Vji%2B%2FOCLdqcE87DvJYEttv9fowLzJi%2BZ3d8P8HL%2BUkD5vCk6%2Ffsx9dZhdrstN5wt5iEBBOR8cYNZWo5q32Wb1DoVFdpKA3%2BkMhZjq8FdHobJGBxGrciJ2W2SnQT9xB%2BjuYhfL7nNnfT0pO8hdjydtDwTF9jkpLVDuXvjaxGUtfX5iXQaCnZK59%2BuUnsmiGKsNyXMAfzZIPpd8j%2F7wbn9oetw%3D%3D)


Больше примеров в отчёте

### Задача 3 (6/10 балла). 

#### Задача 3.1 (4/10 балла) Построить и обучить VAPNEV на CelebA

Здесь нужно прочитать [статью про VAPNEV](https://arxiv.org/pdf/1611.05209.pdf), обучить его до нормального (трешхолды будут позже) качества, померить FID и запомнить для будущего сравнения. 

#### Задача 3.2 (2/10 балла). Построить и обучить conditional VAPNEV на CelebA

Вроде понятно из текста задачи. Что такое conditional VAPNEV, читайте в статье

In [ ]:
# YOUR CODE HERE

### Задача 4. (1/10 балла) Анализ

* Анализ латентного пространства и визуально качественный результат - **1 балл**
1. Посмотрите у какой модели получается лучшая интерполяция в латентном пространстве
2. Попробуйте взять внешнюю картинку (не из селебы), отмапить ее с латентное пространство и покажите ближайшие к ней.
3. Покажите самый смешной результат генерации который у вас получался

In [ ]:
vae = VAE().to(device)
# Последняя версия модели из WandB
vae.load("/content/1080ccad73ee5736290b4ee183f6a2ef")

In [ ]:
with torch.no_grad():
    for scale in [0.2, 0.3, 0.5, 0.7, 0.9, 1, 1.2, 2]:
        show_images(vae.decoder(scale * torch.randn_like(vae.encode(torch.zeros(32, 3, 64, 64).to(device)))).cpu(), title=f'Scale = {scale}')

In [ ]:
with torch.no_grad():
    show_images(vae.decoder(torch.zeros_like(vae.encode(torch.zeros(1, 3, 64, 64).to(device)))).cpu(), title=f'Default person')

In [ ]:
from PIL import Image

In [ ]:
me = transform(Image.open('/content/IMG_3172.png'))
show_images([me], title='eto ya')

In [ ]:
with torch.no_grad():
    target = vae.encode(me.unsqueeze(0).to(device))[0].detach()

closest = (1e9, None)

batch_size = 16

for i, (batch, attr) in tqdm(enumerate(dataloader), total=(len(dataset) + batch_size) // batch_size):
    with torch.no_grad():
        candidates = vae.encode(batch.to(device))
        distances = torch.cdist(candidates, target.view(1, -1))

        min_index = torch.argmin(distances)

        closest = min(closest, (distances[min_index], batch[min_index].clone().detach()))

show_images([closest[-1]])

In [ ]:
sokolov = transform(Image.open('/content/sokolov.jpg'))
show_images([sokolov])

In [ ]:
with torch.no_grad():
    target = vae.encode(sokolov.unsqueeze(0).to(device))[0].detach()

closest = (1e9, None)

batch_size = 16

for i, (batch, attr) in tqdm(enumerate(dataloader), total=(len(dataset) + batch_size) // batch_size):
    with torch.no_grad():
        candidates = vae.encode(batch.to(device))
        distances = torch.cdist(candidates, target.view(1, -1))

        min_index = torch.argmin(distances)

        closest = min(closest, (distances[min_index], batch[min_index].clone().detach()))

show_images([closest[-1]])

### Задача 0. Отчет

Чтобы получить полный балл за каждый из пунктов вам нужно включить в отчет (который сдатеся отдельно в виде pdf) примеры (лучше - много примеров) генерации вашей модели. 

Сравните результаты разных моделей, попробуйте обьяснить почему одна сработала лучше чем другая.

Отчёт прикрепил в отправку + загрущил на диск:
https://disk.yandex.ru/i/27nMc_a1kk6GPA